In [1]:
import pandas as pd

In [26]:
drug_data_for_modeling = pd.read_csv('/home/kdt-admin/miniconda3/envs/JS/WASSUP_Project_Team7_2-1/drug_data_for_h20.csv')

In [32]:
drug_data_for_modeling.head()

,Padding_re_con,encoded_drugName
0,"[7, 10, 4, 1, 494, 2198, 2492, 1197, 312, 3649...",3197
1,"[493, 2555, 1336, 19, 2455, 303, 614, 257, 48,...",1428
2,"[17, 1, 172, 885, 370, 359, 159, 190, 20, 1457...",1851
3,"[15, 13, 17, 475, 14, 12, 469, 50, 278, 15, 35...",2279
4,"[746, 134, 376, 37, 137, 11, 1808, 888, 338, 1...",515


### TPOT

In [37]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

X_tfidf = tfidf_vectorizer.fit_transform(drug_data_for_modeling['Padding_re_con'])

X = X_tfidf
y = drug_data_for_modeling['encoded_drugName']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tpot = TPOTClassifier(config_dict='TPOT sparse', generations=5, population_size=20, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)

print("Test accuracy:", tpot.score(X_test, y_test))

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.23301154321908113

Generation 2 - Current best internal CV score: 0.23301154321908113

Generation 3 - Current best internal CV score: 0.23301154321908113

Generation 4 - Current best internal CV score: 0.23301154321908113

Generation 5 - Current best internal CV score: 0.2859573573539388

Best pipeline: MultinomialNB(SelectPercentile(input_matrix, percentile=16), alpha=0.001, fit_prior=True)
Test accuracy: 0.28333281303655605


In [38]:
tpot.export('drugdata_tpot_pipeline.py')